<a href="https://colab.research.google.com/github/yasaswiyash18/DASHBOARD_DEVELOPMENT/blob/main/CodTech_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash
import pandas as pd
import dash
from dash import dcc, html
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [2]:
# Load dataset, trying 'Orders' as the sheet name
try:
    df = pd.read_excel("Sample - Superstore.xls", sheet_name="Orders")
except ValueError:
    # If 'Orders' doesn't exist, print available sheet names for debugging
    xls = pd.ExcelFile("Sample - Superstore.xls")
    print(f"Available sheets: {xls.sheet_names}")
    raise  # Re-raise the error to stop execution


# Data Preprocessing
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Year'] = df['Order Date'].dt.year

In [3]:
# Initialize Dash App
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Superstore Sales Dashboard", style={'textAlign': 'center'}),

    # Year Selection Dropdown
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': str(year), 'value': year} for year in df['Year'].unique()],
        value=df['Year'].max(),
        clearable=False,
        style={'width': '50%'}
    ),

    # Sales Trend
    dcc.Graph(id='sales-trend'),

    # Category-wise Sales
    dcc.Graph(id='category-sales'),

    # Top Customers
    dcc.Graph(id='top-customers')
])

In [4]:
# Callbacks
@app.callback(
    [Output('sales-trend', 'figure'),
     Output('category-sales', 'figure'),
     Output('top-customers', 'figure')],
    [Input('year-dropdown', 'value')]
)
def update_charts(selected_year):
    filtered_df = df[df['Year'] == selected_year]

    # Sales Trend
    sales_fig = px.line(filtered_df, x='Order Date', y='Sales', title="Sales Over Time", markers=True)

    # Category Sales
    category_fig = px.bar(filtered_df.groupby('Category')['Sales'].sum().reset_index(),
                          x='Category', y='Sales', title="Sales by Category", color='Category')

    # Top Customers
    top_customers = filtered_df.groupby('Customer Name')['Sales'].sum().nlargest(10).reset_index()
    top_cust_fig = px.bar(top_customers, x='Customer Name', y='Sales', title="Top 10 Customers", color='Customer Name')

    return sales_fig, category_fig, top_cust_fig

# Run Server
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>